In [54]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import model_selection
from sklearn.model_selection import train_test_split
import math
import tensorflow as tf
from scipy.sparse import csr_matrix
from fastpivot import pivot_table, pivot_sparse
from sklearn.metrics import pairwise_distances
from surprise import KNNWithMeans
from scipy.sparse import csr_matrix, issparse 
from scipy.stats import pearsonr

In [167]:
movies = pd.read_csv('../datasets/movielens_original/movies.csv')
# movies = pd.read_csv('../datasets/ml-latest-small/movies.csv')
movies.tail()

,movieId,title,genres
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)
62422,209171,Women of Devil's Island (1962),Action|Adventure|Drama


In [56]:
ratings = pd.read_csv('../datasets/movielens_original/ratings.csv')
# ratings = pd.read_csv('../datasets/ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [57]:
df = ratings.merge(movies, how = 'inner', on='movieId',)
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
1,3,296,5.0,1439474476,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
2,4,296,4.0,1573938898,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
3,5,296,4.0,830786155,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
4,7,296,4.0,835444730,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller


In [58]:
model = tf.keras.models.load_model('saved_model.h5')

In [59]:
movies_ids = list(set(list(df.movieId.unique())))
users_ids = list(set(list(df.userId.unique())))

dict_movies = {}
index = 0

for ids in sorted(movies_ids):
    dict_movies[ids] = index
    index += 1

dict_users = {}
index = 0
for ids in sorted(users_ids):
    dict_users[ids] = index
    index += 1


In [82]:
user = 121982
predicted_rating = model.predict([pd.Series([user]*len(dict_movies)), pd.Series(dict_movies.values())])
predicted_rating = pd.DataFrame(predicted_rating)
predicted_rating

1846/1846 [==============================] - 5s 2ms/step


,0
0,4.204201
1,4.342381
2,4.112751
3,4.424030
4,4.388598
...,...
59042,2.528434
59043,4.358899
59044,4.860164
59045,4.120116


In [83]:
# matrix = df.pivot_table(index='userId', columns='title', values = 'rating')

# matrix_norm = matrix.subtract(matrix.mean(axis=1), axis = 'rows')
# user_similarity = matrix_norm.T.corr()
# user_similarity.drop(index=user, inplace=True)
# user_similarity.head()

# user_similarity_cosine = cosine_similarity(matrix_norm.fillna(0), matrix_norm[])
# user_similarity_cosine

# n = 50
# user_similarity_threshold = 0.3

# similar_users = user_similarity[user_similarity[user]>user_similarity_threshold][user].sort_values(ascending=False)[:n]
# similar_users

In [84]:
# matrix = df.pivot_table(index='userId', columns='title', values = 'rating')

# matrix_norm = matrix.subtract(matrix.mean(axis=1), axis = 'rows')
# user_similarity = matrix_norm.T.corr()
# user_similarity.drop(index=user, inplace=True)
# user_similarity.head()

# user_similarity_cosine = cosine_similarity(matrix_norm.fillna(0))
# user_similarity_cosine

# # n = 50
# # user_similarity_threshold = 0.3

# similar_users = user_similarity_cosine[user-1].maximum()
# similar_users

# # similar_users = user_similarity[user_similarity[user]>user_similarity_threshold][user].sort_values(ascending=False)[:n]
# # similar_users

WIlson

In [120]:
def create_X(df): 
    """ 
    Generates a sparse matrix from ratings dataframe. 
        
    Args: 
        df: pandas dataframe 
        
    Returns: 
        X: sparse matrix 
        user_mapper: dict that maps user id's to user indices 
        user_inv_mapper: dict that maps user indices to user id's 
        movie_mapper: dict that maps movie id's to movie indices 
        movie_inv_mapper: dict that maps movie indices to movie id's 
    """ 
    N = ratings['userId'].nunique() 
    M = ratings['movieId'].nunique() 

    user_mapper = dict(zip(np.unique(ratings["userId"]), list(range(N)))) 
    movie_mapper = dict(zip(np.unique(ratings["movieId"]), list(range(M)))) 
        
    user_inv_mapper = dict(zip(list(range(N)), np.unique(df["userId"]))) 
    movie_inv_mapper = dict(zip(list(range(M)), np.unique(df["movieId"]))) 
        
    user_index = [user_mapper[i] for i in df['userId']] 
    movie_index = [movie_mapper[i] for i in df['movieId']] 

    X = csr_matrix((ratings["rating"], (user_index, movie_index)), shape=(N, M)) 
    # X = csr_matrix((ratings["rating"], (movie_index, user_index)), shape=(M, N)) 
        
    return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper

In [121]:
X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_X(ratings)

In [207]:
movie_mapper[59315]

12319

In [206]:
movie_inv_mapper[59315]

KeyError: 59315

In [187]:
movies[movies.movieId == 8360]

,movieId,title,genres
7734,8360,Shrek 2 (2004),Adventure|Animation|Children|Comedy|Musical|Ro...


In [205]:
movies[movies.title.str.contains('Insidious')]

,movieId,title,genres
16228,85788,Insidious (2010),Fantasy|Horror|Thriller
20261,104908,Insidious: Chapter 2 (2013),Horror|Thriller
25054,122884,Insidious: Chapter 3 (2015),Fantasy|Horror|Thriller
51503,183295,Insidious: The Last Key (2018),Horror|Mystery|Thriller


In [122]:
user_movie_matrix = pd.DataFrame.sparse.from_spmatrix(X)
user_movie_matrix

,0,1,2,3,4,5,6,7,8,9,...,59037,59038,59039,59040,59041,59042,59043,59044,59045,59046
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162536,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
162537,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
162538,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
162539,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [123]:
user_movie_matrix.shape

(162541, 59047)

In [153]:
sampled = user_movie_matrix.sample(n = 7000, axis = 0, random_state= 1)
sampled.head()

,0,1,2,3,4,5,6,7,8,9,...,59037,59038,59039,59040,59041,59042,59043,59044,59045,59046
17991,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
616,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50380,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
67657,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [156]:
user = 17991

user_similarity_cosine = cosine_similarity(sampled, pd.DataFrame(sampled.loc[user]).T)
user_similarity_cosine = pd.DataFrame(user_similarity_cosine, index = indexes)
user_similarity_cosine.drop(user, inplace = True)
user_similarity_cosine.head()

,0
616,0.091060
75750,0.018268
50380,0.125491
67657,0.027902
91914,0.254557


In [157]:
indexes = sampled.index

In [158]:
user_df = pd.DataFrame(sampled.loc[user]).T
user_df.head()

,0,1,2,3,4,5,6,7,8,9,...,59037,59038,59039,59040,59041,59042,59043,59044,59045,59046
17991,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [159]:
# Number of similar users
n = 10
# User similarity threashold
user_similarity_threshold = 0.3
# Get top n similar users
similar_users = user_similarity_cosine.sort_values(by = 0, ascending=False)[:n]
# Print out top n similar users
print(f'The similar users for user {user} are', similar_users)

The similar users for user 17991 are                0
112818  0.459821
68449   0.445130
98564   0.434374
104533  0.405509
135557  0.389286
161238  0.383693
37601   0.377685
30647   0.375644
86996   0.375398
36907   0.374845


In [ ]:
# A dictionary to store item scores
item_score = {}
# Loop through items
for i in sampled.columns:
  # Get the ratings for movie i
  movie_rating = sampled[i]
  # Create a variable to store the score
  total = 0
  # Create a variable to store the number of scores
  count = 0
  # Loop through similar users
  for u in similar_users.index:
    # If the movie has rating
    if pd.isna(movie_rating[u]) == False:
      # Score is the sum of user similarity score multiply by the movie rating
      score = similar_users[u] * movie_rating[u]
      # Add the score to the total score for the movie so far
      total += score
      # Add 1 to the count
      count +=1
  # Get the average score for the item
  item_score[i] = total / count
# Convert dictionary to pandas dataframe
item_score = pd.DataFrame(item_score.items(), columns=['movie', 'movie_score'])
    
# Sort the movies by score
ranked_item_score = item_score.sort_values(by='movie_score', ascending=False)
# Select top m movies
m = 10
ranked_item_score.head(m)

KeyError: 140963